<IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">

    
# Local grid refinement   

*O.N. Ebbens, Artesia, 2022*

This notebook shows an example of a vertex model create with the `nlmod` package.

### Contents<a name="TOC"></a>
1. [Create model](#create)
2. [Local grid refinement](#lgr)
3. [Run Model](#run)
3. [Visualise](#Visualise)

In [ ]:
import logging
import os

import flopy
import geopandas as gpd
import matplotlib.pyplot as plt
import nlmod


In [ ]:
print(f'nlmod version: {nlmod.__version__}')

# toon informatie bij het aanroepen van functies
logging.basicConfig(level=logging.INFO)

### [1. Create model](#TOC)<a name="create"></a>

Modflow 6 makes it possible to use locally refined grids. In NLmod you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See also figures below.

In [ ]:
# model settings vertex
model_ws = "model3"
model_name = "IJm_planeten"
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
refine_shp_fname = os.path.join("data", "planetenweg_ijmuiden")
levels = 2
extent = [95000.0, 105000.0, 494000.0, 500000.0]
delr = 100.0
delc = 100.0
steady_state = False
steady_start = True
transient_timesteps = 5
perlen = 1.0
start_time = "2015-1-1"
use_regis = True
regis_botm_layer = "MSz1"
use_geotop = True
add_northsea = True
starting_head = 1.0


In [ ]:
layer_model = nlmod.read.regis.get_combined_layer_models(
    extent,
    use_regis=use_regis,
    regis_botm_layer=regis_botm_layer,
    use_geotop=use_geotop,
    cachedir=cachedir,
    cachename="combined_layer_ds.nc",
)

# create a model ds by changing grid of layer_model
ds = nlmod.mdims.to_model_ds(
    layer_model, model_name, model_ws, delr=delr, delc=delc
)

# add time discretisation
ds = nlmod.mdims.set_ds_time(
    ds,
    start_time=start_time,
    steady_state=steady_state,
    steady_start=steady_start,
    transient_timesteps=transient_timesteps,
    perlen=perlen,
)


### [2. Local grid refinement](#TOC)<a name="lgr"></a>

the code below applies a local grid refinement to the layer model. The local grid refinement is based on the shapefile 'planetenweg_ijmuiden.shp', which contains a line shape of the Planetenweg, and the levels, which is 2. This means that the model cells at the Planetenweg will get a size of 25 x 25m. 

In [ ]:
# use gridgen to create vertex grid
ds = nlmod.mgrid.refine(
    ds, refinement_features=[(refine_shp_fname, "line", levels)]
)

if add_northsea:
    ds = nlmod.mdims.add_northsea(ds, cachedir=cachedir)


In [ ]:
# create simulation
sim = nlmod.sim.sim(ds)

# create time discretisation
tdis = nlmod.sim.tdis(ds, sim)

# create groundwater flow model
gwf = nlmod.gwf.gwf(ds, sim)

# create ims
ims = nlmod.gwf.ims(sim)

# Create discretization
disv = nlmod.gwf.disv(ds, gwf)

# create node property flow
npf = nlmod.gwf.npf(ds, gwf, save_flows=True)

# Create the initial conditions package
ic = nlmod.gwf.ic(ds, gwf, starting_head=starting_head)

# Create the output control package
oc = nlmod.gwf.oc(ds, gwf)


In [ ]:
# voeg grote oppervlaktewaterlichamen toe
da_name = "rws_oppwater"
rws_ds = nlmod.read.rws.get_surface_water(
    ds, da_name, cachedir=ds.cachedir, cachename=da_name + ".nc"
)
ds.update(rws_ds)
ghb = nlmod.gwf.ghb(ds, gwf, da_name)

# surface level drain
ahn_ds = nlmod.read.ahn.get_ahn(ds, cachedir=ds.cachedir, cachename="ahn.nc")
ds.update(ahn_ds)

drn = nlmod.gwf.surface_drain_from_ds(ds, gwf)


# add constant head cells at model boundaries
ds.update(nlmod.gwf.constant_head.chd_at_model_edge(ds, ds["idomain"]))
chd = nlmod.gwf.chd(ds, gwf, head="starting_head")


In [ ]:
# add knmi recharge to the model datasets
knmi_ds = nlmod.read.knmi.get_recharge(
    ds, cachedir=ds.cachedir, cachename="recharge"
)
ds.update(knmi_ds)

# create recharge package
rch = nlmod.gwf.rch(ds, gwf)


In [ ]:
ds

### [3. Write and Run](#TOC)<a name="run"></a>


In [ ]:
nlmod.sim.write_and_run(
    sim, ds, write_ds=True, nb_path="03_local_grid_refinement.ipynb"
)


### [4. Visualise](#TOC)<a name="visualise"></a>

Using the `ds` and `gwf` variables it is quite easy to visualise model data. Below the modelgrid together with the surface water is shown.

In [ ]:
plan_weg_gdf = gpd.read_file(refine_shp_fname + ".shp")

# plot modelgrid
ax = nlmod.visualise.plots.plot_modelgrid(ds, gwf)
plan_weg_gdf.plot(ax=ax, color="r", label="Planetenweg")
ax.legend()

# plot zoomed modelgrid
ax = nlmod.visualise.plots.plot_modelgrid(ds, gwf)
ax.set_title("Planetenweg")
plan_weg_gdf.plot(ax=ax, color="r", label="Planetenweg")
ax.set_xlim(100000, 103000)
ax.set_ylim(495000, 497500)
ax.legend()

The model dataset of a vertex model differs from a structured model dataset. The data is stored relative to the cell-id instead of the row and column number. Therefore the model dataset has the dimension icell2d instead of the dimensions x and y. 

In [ ]:
ds

To plot the same rasters as for the previous model we can use the `nlmod.visualise.plots.plot_vertex_array()` function.

In [ ]:
ds["vertices"] = nlmod.mdims.get_vertices(ds)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 11))
nlmod.visualise.plots.plot_vertex_array(
    ds["ahn"], ds["vertices"], ax=axes[0][0]
)
nlmod.visualise.plots.plot_vertex_array(
    ds["botm"][0], ds["vertices"], ax=axes[0][1]
)
nlmod.visualise.plots.plot_vertex_array(
    ds["idomain"][0], ds["vertices"], ax=axes[1][0]
)
nlmod.visualise.plots.plot_vertex_array(
    ds["chd"][0], ds["vertices"], ax=axes[1][1]
)
for axes1 in axes:
    for ax in axes1:
        ax.axis("scaled")

fig.savefig(
    os.path.join(ds.figdir, "ahn_bot_idom_chd.png"), bbox_inches="tight"
)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 11))
nlmod.visualise.plots.plot_vertex_array(
    ds["bathymetry"], ds["vertices"], ax=axes[0][0]
)
nlmod.visualise.plots.plot_vertex_array(
    ds["northsea"], ds["vertices"], ax=axes[0][1]
)
nlmod.visualise.plots.plot_vertex_array(
    ds["kh"][1], ds["vertices"], ax=axes[1][0]
)
nlmod.visualise.plots.plot_vertex_array(
    ds["recharge"][:, 0], ds["vertices"], ax=axes[1][1]
)

for axes1 in axes:
    for ax in axes1:
        ax.axis("scaled")
fig.savefig(
    os.path.join(ds.figdir, "bath_nsea_kh_rch.png"), bbox_inches="tight"
)


We can save the entire model as a UGRID NetCDF-file. This can be opened in qgis, as a 'Mesh Layer'. For more information see https://docs.qgis.org/3.16/en/docs/user_manual/working_with_mesh/mesh_properties.html

In [ ]:
fname = os.path.join(ds.figdir, 'results.nc')
nlmod.visualise.gis.model_dataset_to_ugrid_nc_file(ds, fname)